In [1]:
import pandas as pd
from pandas.plotting import table
import numpy as np  
import matplotlib.pyplot as plt
import openpyxl
from IPython.display import HTML
import pingouin as pg
import os
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [2]:
xl_usu = 'usu_individual_T125.xlsx'  
dfusu = pd.read_excel(xl_usu)
dfusu = dfusu.loc[:, ['CODUSU','NRO_HOGAR','COMPONENTE', 'ESTADO','V5_01_M','V5_02_M','V5_03_M','PONDERA']]

In [3]:
xl_hog = 'usu_hogar_T125.xlsx'  
dfhog = pd.read_excel(xl_hog)
dfhog = dfhog.loc[:, ['CODUSU','NRO_HOGAR','DECIFR','REGION','V5_01','V5_02','V5_03']]

In [4]:
# V5 = 1 si ALGUNA variable es 1, 2 solo si TODAS son 2
dfhog['V5'] = 2  # Inicializar con 2 (asumiendo que todas son 2)

# Si ALGUNA variable es 1, entonces V5 = 1
mask_alguna_1 = (dfhog['V5_01'] == 1) | (dfhog['V5_02'] == 1) | (dfhog['V5_03'] == 1)
dfhog.loc[mask_alguna_1, 'V5'] = 1

#print(dfhog['V5'].value_counts().sort_index())
dfhog.head()

,CODUSU,NRO_HOGAR,DECIFR,REGION,V5_01,V5_02,V5_03,V5
0,TQRMNOQVSHKOLNCDEGGFB00858441,1,1,42,2,2,2,2
1,TQSMNORTSHMOLTCDEGGFB00877605,1,6,42,2,2,2,2
2,TQRMNOQQXHMMMLCDEGGFB00877606,1,4,42,2,2,2,2
3,TQRMNOTTWHMMLOCDEFIAH00877819,1,12,43,2,2,2,2
4,TQRMNORQQHLMKOCDEHIBB00853810,2,1,40,2,2,2,2


In [5]:
#hogares: filtrar 50% más pobre (0 y 12)

dfhog_pobre = dfhog[dfhog['DECIFR'].isin([1, 2, 3, 4, 5])]
print(dfhog_pobre['DECIFR'].value_counts())

#persona: filtrar por Estado 1 (ocupado), 2 (desocupado) y 3 (inactivo)
dfusu = dfusu[dfusu['ESTADO'].isin([1, 2, 3])]
print(dfusu['ESTADO'].value_counts())

print("---------------------------------------------------")
dfhog_pobre.info()
dfusu.info()

DECIFR
1    1457
4    1350
2    1333
5    1287
3    1273
Name: count, dtype: int64
ESTADO
1    20076
3    18866
2     1381
Name: count, dtype: int64
---------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 6700 entries, 0 to 15979
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CODUSU     6700 non-null   object
 1   NRO_HOGAR  6700 non-null   int64 
 2   DECIFR     6700 non-null   int64 
 3   REGION     6700 non-null   int64 
 4   V5_01      6700 non-null   int64 
 5   V5_02      6700 non-null   int64 
 6   V5_03      6700 non-null   int64 
 7   V5         6700 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 471.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 40323 entries, 0 to 45424
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CODUSU      40323 non-null  object
 1   NRO_HOGAR   40323 non-null 

In [6]:
#Juntar tablas (left hogares pobres, right individuos)
df_join_pobre = dfhog_pobre.merge(dfusu, how='left', on=['CODUSU', 'NRO_HOGAR'])
df_join_pobre.info()
df_join_pobre.head(20)  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14030 entries, 0 to 14029
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CODUSU      14030 non-null  object
 1   NRO_HOGAR   14030 non-null  int64 
 2   DECIFR      14030 non-null  int64 
 3   REGION      14030 non-null  int64 
 4   V5_01       14030 non-null  int64 
 5   V5_02       14030 non-null  int64 
 6   V5_03       14030 non-null  int64 
 7   V5          14030 non-null  int64 
 8   COMPONENTE  14030 non-null  int64 
 9   ESTADO      14030 non-null  int64 
 10  V5_01_M     14030 non-null  int64 
 11  V5_02_M     14030 non-null  int64 
 12  V5_03_M     14030 non-null  int64 
 13  PONDERA     14030 non-null  int64 
dtypes: int64(13), object(1)
memory usage: 1.5+ MB


,CODUSU,NRO_HOGAR,DECIFR,REGION,V5_01,V5_02,V5_03,V5,COMPONENTE,ESTADO,V5_01_M,V5_02_M,V5_03_M,PONDERA
0,TQRMNOQVSHKOLNCDEGGFB00858441,1,1,42,2,2,2,2,1,1,0,0,0,526
1,TQRMNOQVSHKOLNCDEGGFB00858441,1,1,42,2,2,2,2,2,3,0,0,0,526
2,TQRMNOQQXHMMMLCDEGGFB00877606,1,4,42,2,2,2,2,1,3,0,0,0,446
3,TQRMNOQQXHMMMLCDEGGFB00877606,1,4,42,2,2,2,2,2,3,0,0,0,446
4,TQRMNORQQHLMKOCDEHIBB00853810,2,1,40,2,2,2,2,1,3,0,0,0,132
5,TQRMNOTPYHMOLQCDEFMDB00877823,1,5,43,2,2,2,2,1,1,0,0,0,199
6,TQRMNOTPYHMOLQCDEFMDB00877823,1,5,43,2,2,2,2,2,1,0,0,0,199
7,TQRMNORVVHLLLPCDEFMDB00852190,1,3,43,2,2,2,2,1,3,0,0,0,146
8,TQRMNOQTTHKMKSCDEFMDB00857883,1,3,43,2,2,2,2,1,1,0,0,0,234
9,TQRMNOPXWHMOKSCDEOJAH00877825,1,3,44,2,2,2,2,1,1,0,0,0,113


In [7]:
# Crear UID único para el DataFrame completo
df_join_pobre['UID'] = (
    df_join_pobre['CODUSU'].astype(str) + '_' + 
    df_join_pobre['NRO_HOGAR'].astype(str) + '_' + 
    df_join_pobre['COMPONENTE'].astype(str)
)   
df_join_pobre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14030 entries, 0 to 14029
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CODUSU      14030 non-null  object
 1   NRO_HOGAR   14030 non-null  int64 
 2   DECIFR      14030 non-null  int64 
 3   REGION      14030 non-null  int64 
 4   V5_01       14030 non-null  int64 
 5   V5_02       14030 non-null  int64 
 6   V5_03       14030 non-null  int64 
 7   V5          14030 non-null  int64 
 8   COMPONENTE  14030 non-null  int64 
 9   ESTADO      14030 non-null  int64 
 10  V5_01_M     14030 non-null  int64 
 11  V5_02_M     14030 non-null  int64 
 12  V5_03_M     14030 non-null  int64 
 13  PONDERA     14030 non-null  int64 
 14  UID         14030 non-null  object
dtypes: int64(13), object(2)
memory usage: 1.6+ MB


In [8]:
# Recodificar variables (fusionar 1 y 2 como Activo; v5 con percibe/no-percibe)
# Creamos nuevas columnas con sufijo _CAT para mantener las originales
df_join_pobre['ESTADO_CAT'] = df_join_pobre['ESTADO'].replace({1: 'Activo', 2: 'Activo', 3: 'Inactivo'})
df_join_pobre['V5_CAT'] = df_join_pobre['V5'].replace({1: 'Percibe', 2: 'No Percibe'})
df_join_pobre['REGION_CAT'] = df_join_pobre['REGION'].replace({1: 'Gran Buenos Aires', 40: 'Noroeste', 41: 'Noreste', 42: 'Cuyo', 43: 'Pampeana', 44: 'Patagonia'})

print("Variables recodificadas con sufijo _CAT.")
df_join_pobre[['ESTADO_CAT', 'V5_CAT', 'REGION_CAT']].head()

Variables recodificadas con sufijo _CAT.


,ESTADO_CAT,V5_CAT,REGION_CAT
0,Activo,No Percibe,Cuyo
1,Inactivo,No Percibe,Cuyo
2,Inactivo,No Percibe,Cuyo
3,Inactivo,No Percibe,Cuyo
4,Inactivo,No Percibe,Noroeste


In [9]:
#ponderador casero
n = len(df_join_pobre)
N_t = df_join_pobre['PONDERA'].sum()
ajuste = n / N_t
df_join_pobre['PONDERA2'] = df_join_pobre['PONDERA'] * ajuste
#verif ponderadores
print(n)
print(df_join_pobre['PONDERA'].sum())
print(df_join_pobre['PONDERA2'].sum())

14030
8225518
14030.0


In [15]:
# --- Preparación para el Excel ---
# Definimos el nombre del archivo y creamos el "escritor" de Excel.
# Dejamos el writer "abierto" para agregarle pestañas en las celdas siguientes.
output_filename = 'Output_EPH_Transferencias_Actividad.xlsx'
writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')

# --- Función de Ayuda para Chi-Cuadrado ---
def get_chi2_results(abs_table):
    """Calcula Chi² y devuelve un string formateado."""
    table_no_totals = abs_table.iloc[:-1, :-1]
    if table_no_totals.empty or table_no_totals.sum().sum() == 0:
        return "Datos insuficientes"
    try:
        chi2, p_valor, _, _ = chi2_contingency(table_no_totals)
        p_str = f"p={p_valor:.4f}"
        if p_valor < 0.05:
            p_str += " (*)"
        return f"Chi²={chi2:.2f}; {p_str}"
    except ValueError as e:
        return f"Cálculo no válido ({e})"

print(f"Writer de Excel preparado para guardar en: {output_filename}")

Writer de Excel preparado para guardar en: Output_EPH_Transferencias_Actividad.xlsx


In [16]:
# --- PESTAÑA 1: ANÁLISIS TABULAR (CHI-CUADRADO) ---
print("Generando Pestaña 1: 'Analisis_Tabular'...")
sheet_name_chi2 = 'Analisis_Tabular'

# Calcular dónde irá la columna de Chi²
index_cols_count = 3 # REGION_CAT, ESTADO_CAT, Tipo
data_cols_count = df_join_pobre['V5_CAT'].nunique() + 1 # Percibe, No Percibe, Total
col_inicio_chi2 = index_cols_count + data_cols_count + 1
fila_actual = 0

# Escribir el título de la columna Chi²
pd.DataFrame(['Análisis Chi² (χ²)']).to_excel(
    writer, sheet_name=sheet_name_chi2, 
    startrow=fila_actual, startcol=col_inicio_chi2, 
    header=False, index=False
)
fila_actual += 2

# --- 1. TABLA NACIONAL (TOTAL) ---
tabla_nac_abs = pd.pivot_table(
    df_join_pobre, values='PONDERA2', index='ESTADO_CAT', columns='V5_CAT',
    aggfunc='sum', margins=True, margins_name='Total', fill_value=0
)
tabla_nac_pct = tabla_nac_abs.apply(lambda x: (x / x['Total']) * 100).round(2)
chi2_nac_str = get_chi2_results(tabla_nac_abs)

tabla_nac_abs['Tipo'] = 'Absoluto'
tabla_nac_pct['Tipo'] = 'Porcentaje'
tabla_nacional_final = pd.concat([tabla_nac_abs, tabla_nac_pct]).set_index('Tipo', append=True)
tabla_nacional_listo = pd.concat([tabla_nacional_final], keys=['Total (Nacional)'], names=['REGION_CAT'])
tabla_nacional_listo = tabla_nacional_listo.sort_index()

tabla_nacional_listo.to_excel(writer, sheet_name=sheet_name_chi2, startrow=fila_actual)

fila_chi2 = fila_actual + (tabla_nacional_listo.shape[0] // 2)
pd.DataFrame([chi2_nac_str]).to_excel(
    writer, sheet_name=sheet_name_chi2, 
    startrow=fila_chi2, startcol=col_inicio_chi2, 
    header=False, index=False
)
fila_actual += tabla_nacional_listo.shape[0] + 2

# --- 2. TABLAS PARCIALES (POR REGION) ---
regiones_ordenadas = sorted(df_join_pobre['REGION_CAT'].unique())

for region_id in regiones_ordenadas:
    df_region = df_join_pobre[df_join_pobre['REGION_CAT'] == region_id]
    if df_region.empty:
        continue
    
    tabla_reg_abs = pd.pivot_table(
        df_region, values='PONDERA2', index='ESTADO_CAT', columns='V5_CAT',
        aggfunc='sum', margins=True, margins_name='Total', fill_value=0
    )
    tabla_reg_pct = tabla_reg_abs.apply(lambda x: (x / x['Total']) * 100).round(2)
    chi2_reg_str = get_chi2_results(tabla_reg_abs)
    
    tabla_reg_abs['Tipo'] = 'Absoluto'
    tabla_reg_pct['Tipo'] = 'Porcentaje'
    tabla_region_final = pd.concat([tabla_reg_abs, tabla_reg_pct]).set_index('Tipo', append=True)
    
    tabla_region_listo = pd.concat([tabla_region_final], keys=[region_id], names=['REGION_CAT'])
    tabla_region_listo = tabla_region_listo.sort_index()
    
    tabla_region_listo.to_excel(
        writer, sheet_name=sheet_name_chi2, 
        startrow=fila_actual, 
        header=False # El encabezado ya se escribió con la tabla nacional
    )
    
    fila_chi2 = fila_actual + (tabla_region_listo.shape[0] // 2)
    pd.DataFrame([chi2_reg_str]).to_excel(
        writer, sheet_name=sheet_name_chi2, 
        startrow=fila_chi2, startcol=col_inicio_chi2, 
        header=False, index=False
    )
    fila_actual += tabla_region_listo.shape[0] + 2

print("Pestaña 'Analisis_Tabular' generada.")

Generando Pestaña 1: 'Analisis_Tabular'...
Pestaña 'Analisis_Tabular' generada.


In [17]:
# --- PESTAÑA 2: ANÁLISIS DE REGRESIÓN LOGÍSTICA (COEFICIENTES) ---
print("Generando Pestaña 2: 'Analisis_Regresion'...")
sheet_name_reg = 'Analisis_Regresion'

# --- Preparación para la Regresión ---

# 1. Crear variable dependiente Binaria (1=Activo, 0=Inactivo)
df_join_pobre['ESTADO_BIN'] = df_join_pobre['ESTADO_CAT'].apply(lambda x: 1 if x == 'Activo' else 0)

# 2. Función de Ayuda para R² de Nagelkerke
def get_nagelkerke_r2(results):
    """Calcula el Pseudo R² de Nagelkerke a partir de un resultado de statsmodels."""
    try:
        llf = results.llf  # Log-likelihood del modelo completo
        lln = results.llnull # Log-likelihood del modelo nulo (solo intercepto)
        n = results.nobs   # Número de observaciones (suma de ponderadores)
        
        r2_cs = 1 - (np.exp(lln - llf))**(2/n)
        r2_max_nagel = 1 - (np.exp(lln))**(2/n)
        
        if r2_max_nagel == 0: return 0.0
        return r2_cs / r2_max_nagel
    except Exception:
        return np.nan

# 3. Lista para guardar todos los resultados de regresión
all_regression_results = []

# --- Modelo 1: REGRESIÓN LOGÍSTICA MÚLTIPLE (NACIONAL) ---
print("  Calculando: Regresión Logística Múltiple (Nacional)...")

try:
    df_nat = df_join_pobre
    pesos_nat = df_nat['PONDERA2']
    
    # Usamos C() para asegurar que statsmodels trate V5_CAT y REGION_CAT como categóricas
    formula_multi = 'ESTADO_BIN ~ C(V5_CAT) + C(REGION_CAT)'
    
    # Guardamos el modelo en 'modelo_nat' para usarlo en la celda siguiente
    modelo_nat = smf.glm(formula=formula_multi, 
                         data=df_nat, 
                         family=sm.families.Binomial(), 
                         freq_weights=pesos_nat).fit()
    
    # Extraer resultados y ponerlos en un DataFrame
    results_df = pd.DataFrame({
        'Coef. (Beta)': modelo_nat.params,
        'Odds Ratio (e^Beta)': np.exp(modelo_nat.params), # Beta Exponenciado
        'Error Estándar': modelo_nat.bse,
        'P-Valor': modelo_nat.pvalues
    })
    
    # Calcular Nagelkerke R² para el modelo general
    nagel_nat = get_nagelkerke_r2(modelo_nat)
    results_df.loc['Nagelkerke R²', 'Coef. (Beta)'] = nagel_nat
    results_df.loc['N (Ponderado)', 'Coef. (Beta)'] = modelo_nat.nobs
    
    # Añadir identificadores al índice
    results_df['Modelo'] = 'Logístico Múltiple (Nacional)'
    results_df['Estrato'] = 'Total (Nacional)'
    results_df = results_df.reset_index().rename(columns={'index': 'Variable'})
    
    all_regression_results.append(results_df)

except Exception as e:
    print(f"  ERROR al calcular modelo nacional: {e}")

# --- Modelo 2: ANÁLISIS ESTRATIFICADO (REGRESIONES SIMPLES POR REGIÓN) ---
print("  Calculando: Análisis Estratificado (Simple por Región)...")

formula_simple = 'ESTADO_BIN ~ C(V5_CAT)'
regiones_ordenadas = sorted(df_join_pobre['REGION_CAT'].unique())

# Diccionario para guardar los modelos regionales para la celda siguiente
modelos_regionales = {}

for region_id in regiones_ordenadas:
    df_region = df_join_pobre[df_join_pobre['REGION_CAT'] == region_id]
    
    if df_region.empty or df_region['V5_CAT'].nunique() < 2 or df_region['ESTADO_BIN'].nunique() < 2:
        print(f"  > Omitiendo Región {region_id}: datos insuficientes.")
        continue
        
    try:
        pesos_reg = df_region['PONDERA2']
        modelo_reg = smf.glm(formula=formula_simple, 
                             data=df_region, 
                             family=sm.families.Binomial(), 
                             freq_weights=pesos_reg).fit()
        
        # Guardamos el modelo regional
        modelos_regionales[region_id] = modelo_reg
        
        # Extraer resultados
        results_df_reg = pd.DataFrame({
            'Coef. (Beta)': modelo_reg.params,
            'Odds Ratio (e^Beta)': np.exp(modelo_reg.params), # Beta Exponenciado
            'Error Estándar': modelo_reg.bse,
            'P-Valor': modelo_reg.pvalues
        })
        
        nagel_reg = get_nagelkerke_r2(modelo_reg)
        results_df_reg.loc['Nagelkerke R²', 'Coef. (Beta)'] = nagel_reg
        results_df_reg.loc['N (Ponderado)', 'Coef. (Beta)'] = modelo_reg.nobs
        
        results_df_reg['Modelo'] = 'Logístico Simple (Estratificado)'
        results_df_reg['Estrato'] = region_id
        results_df_reg = results_df_reg.reset_index().rename(columns={'index': 'Variable'})
        
        all_regression_results.append(results_df_reg)
        print(f"  > Región {region_id} calculada.")
        
    except Exception as e:
        print(f"  > ERROR al calcular modelo para {region_id}: {e}")

# --- Guardar Pestaña 2 ---
if all_regression_results:
    # Combinar todos los resultados en un solo DataFrame
    df_all_regressions = pd.concat(all_regression_results, ignore_index=True)
    
    # Reordenar columnas para mejor lectura
    column_order = ['Modelo', 'Estrato', 'Variable', 'Coef. (Beta)', 'Odds Ratio (e^Beta)', 'Error Estándar', 'P-Valor']
    df_all_regressions = df_all_regressions[column_order]
    
    # Formatear números
    df_all_regressions['Coef. (Beta)'] = df_all_regressions['Coef. (Beta)'].round(4)
    df_all_regressions['Odds Ratio (e^Beta)'] = df_all_regressions['Odds Ratio (e^Beta)'].round(4)
    df_all_regressions['Error Estándar'] = df_all_regressions['Error Estándar'].round(4)
    df_all_regressions['P-Valor'] = df_all_regressions['P-Valor'].map(lambda x: f"{x:.4f}" if isinstance(x, (int, float)) and not np.isnan(x) else "")
    
    # Escribir a la pestaña de regresión
    df_all_regressions.to_excel(writer, sheet_name=sheet_name_reg, index=False)
    print(f"Pestaña '{sheet_name_reg}' generada.")
else:
    print("No se generaron resultados de regresión.")

Generando Pestaña 2: 'Analisis_Regresion'...
  Calculando: Regresión Logística Múltiple (Nacional)...
  Calculando: Análisis Estratificado (Simple por Región)...
  > Región Cuyo calculada.
  > Región Gran Buenos Aires calculada.
  > Región Noreste calculada.
  > Región Noroeste calculada.
  > Región Pampeana calculada.
  > Región Patagonia calculada.
Pestaña 'Analisis_Regresion' generada.


In [18]:
# --- PESTAÑA 3: EVALUACIÓN DE MODELOS (MATRIZ DE CONFUSIÓN) ---

# Importamos la función necesaria
from sklearn.metrics import confusion_matrix
import numpy as np

print("Generando Pestaña 3: 'Evaluacion_Modelo'...")
sheet_name_eval = 'Evaluacion_Modelo'
fila_inicio = 0

# --- 1. Evaluación Modelo Múltiple (Nacional) ---
print("  Calculando Matriz de Confusión (Nacional)...")

try:
    # Verificamos que 'modelo_nat' exista (creado en la celda anterior)
    if 'modelo_nat' in locals():
        df_nat = df_join_pobre
        y_actual = df_nat['ESTADO_BIN']
        y_pred_prob = modelo_nat.predict(df_nat)
        y_pred_clase = (y_pred_prob > 0.5).astype(int) # Corte en 0.5
        pesos = df_nat['PONDERA2']
        
        # Calcular la matriz de confusión PONDERADA
        cm = confusion_matrix(y_actual, y_pred_clase, sample_weight=pesos)
        
        # Extraer VP, VN, FP, FN (cm[fila, col])
        # Fila 0 = Realidad Inactivo(0), Fila 1 = Realidad Activo(1)
        # Col 0 = Pred Inactivo(0), Col 1 = Pred Activo(1)
        VN = cm[0, 0]
        FP = cm[0, 1]
        FN = cm[1, 0]
        VP = cm[1, 1]
        
        Total = VN + FP + FN + VP
        
        # Calcular Métricas (evitando división por cero si un denominador es 0)
        Accuracy = (VP + VN) / Total if Total > 0 else 0
        Sensitivity = VP / (VP + FN) if (VP + FN) > 0 else 0
        Specificity = VN / (VN + FP) if (VN + FP) > 0 else 0
        FPR = FP / (VN + FP) if (VN + FP) > 0 else 0 # Tasa de Falsos Positivos
        Precision = VP / (VP + FP) if (VP + FP) > 0 else 0
        
        # --- Escribir al Excel ---
        
        # Título de la Matriz
        pd.DataFrame(['Matriz de Confusión - Modelo Múltiple (Nacional) (Ponderada)']).to_excel(
            writer, sheet_name=sheet_name_eval, startrow=fila_inicio, index=False, header=False
        )
        fila_inicio += 2
        
        # DataFrame de Matriz de Confusión
        matrix_df = pd.DataFrame(
            [[VN, FP], [FN, VP]],
            columns=pd.Index(['Predijo: Inactivo (0)', 'Predijo: Activo (1)'], name='Predicción'),
            index=pd.Index(['Real: Inactivo (0)', 'Real: Activo (1)'], name='Realidad')
        )
        matrix_df.loc['Total_Pred'] = matrix_df.sum().values
        matrix_df['Total_Real'] = matrix_df.sum(axis=1).values
        
        matrix_df.to_excel(writer, sheet_name=sheet_name_eval, startrow=fila_inicio)
        fila_inicio += matrix_df.shape[0] + 3 # Espacio
        
        # Título de las Métricas
        pd.DataFrame(['Métricas de Evaluación - Modelo Múltiple (Nacional) (Ponderada)']).to_excel(
            writer, sheet_name=sheet_name_eval, startrow=fila_inicio, index=False, header=False
        )
        fila_inicio += 2
        
        # DataFrame de Métricas
        metrics_df = pd.DataFrame({
            'Metrica': [
                'Verdaderos Positivos (VP)', 'Verdaderos Negativos (VN)',
                'Falsos Positivos (FP)', 'Falsos Negativos (FN)',
                'Total (N Ponderado)',
                'Accuracy (Exactitud)',
                'Sensitivity (Tasa VP / Recall)',
                'Specificity (Tasa VN)',
                'False Positive Rate (Tasa FP)',
                'Precision'
            ],
            'Valor': [
                VP, VN, FP, FN, Total,
                Accuracy, Sensitivity, Specificity, FPR, Precision
            ]
        })
        metrics_df['Valor'] = metrics_df['Valor'].round(4)
        
        metrics_df.to_excel(writer, sheet_name=sheet_name_eval, startrow=fila_inicio, index=False)
        
        print("  > Matriz Nacional y Métricas guardadas.")

    else:
        print("  > 'modelo_nat' no encontrado. Asegúrate de correr la celda 12.")

except Exception as e:
    print(f"  > ERROR al calcular matriz de confusión nacional: {e}")

# --- CERRAR Y GUARDAR EL ARCHIVO EXCEL ---
# Este es el paso final que guarda el archivo con las 3 pestañas.
print("\nCerrando y guardando el archivo Excel...")
writer.close()
print(f"¡Listo! Archivo '{output_filename}' guardado con éxito con 3 pestañas.")

Generando Pestaña 3: 'Evaluacion_Modelo'...
  Calculando Matriz de Confusión (Nacional)...
  > Matriz Nacional y Métricas guardadas.

Cerrando y guardando el archivo Excel...
¡Listo! Archivo 'Output_EPH_Transferencias_Actividad.xlsx' guardado con éxito con 3 pestañas.
